In [1]:
# import libraries
import pickle
import pandas as pd

In [2]:
#  parameters
year = 2022
month = 2
taxi_type = 'yellow'

input_file = f"https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet"
output_file = f"output/{taxi_type}/{year:04d}-{month:02d}.parquet"

In [3]:
# load pickeled model
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [4]:
# preprocess data
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
# Read input file
df = read_data(input_file)

In [6]:
# predict
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [7]:
# save above y_pred array to pandas dataframe
df_y_pred = pd.DataFrame(data=y_pred, columns=['predicted'])

In [8]:
# What's the standard deviation of the predicted duration for this df?
print('std dev of predicted duration:', df_y_pred['predicted'].std())

std dev of predicted duration: 5.281404481465351


In [9]:
# What's the mean of the predicted duration for this df?
print('Mean of predicted duration:', df_y_pred['predicted'].mean())

Mean of predicted duration: 12.513422116701408


In [10]:
df_y_pred.describe()

,predicted
count,2.918187e+06
mean,1.251342e+01
std,5.281404e+00
min,-7.485578e+00
25%,9.945231e+00
50%,1.066465e+01
75%,1.213280e+01
max,8.013667e+01


###  Preparing the output

In [11]:
# let's create an artificial ride_id column
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
# create empty results df
df_result = pd.DataFrame()
#  add columns
df_result['ride_id'] = df['ride_id']
df_result['predictions'] = df_y_pred


In [12]:
# create output and yellow dir, if it does not exist
# !mkdir output
# !mkdir output/yellow

In [13]:
# !ls output

In [14]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [15]:
# check size of file
!ls -lh output/yellow/

total 58M
-rw-rw-r-- 1 ranga ranga 58M Jun 26 00:43 2022-02.parquet
